In [ ]:
# Installing the Googlemaps Package
!pip install googlemaps

In [2]:
# Importing Required Packages
import requests
import pandas as pd
from IPython.display import display, HTML
from google.colab import files
import googlemaps

In [3]:
# Uploading the Yelp API Key
upload = files.upload()

Saving Yelp_API_Key to Yelp_API_Key


In [4]:
# Reading the File and Storing it as a Variable
with open('Yelp_API_Key') as file:
    Yelp_API_Key = file.read().strip("'")

In [5]:
# Uploading the Google API Key
upload = files.upload()

Saving Google_API_Key to Google_API_Key


In [6]:
# Reading the File and Storing it as a Variable
with open('Google_API_Key', 'r') as file:
    google_API_Key = file.read().strip("'")

In [7]:
# Creating a GoogleMap Client
gmaps = googlemaps.Client(key=google_API_Key)

In [8]:
# Function to Create a Scrollable Table Within a Small Window
def scroll_table(data, table_id, title):
    html = f'<h3>{title}</h3>'
    html += f'<div id="{table_id}" style="height:400px;overflow:auto;">'
    html += data.to_html()
    html += '</div>'
    return html

In [9]:
# Setting API Key, URL, and Authorization Variables
search_url = "https://api.yelp.com/v3/businesses/search?sort_by=best_match&limit=50"
Authorization = {'Authorization': 'bearer %s' % Yelp_API_Key}

In [10]:
# Obtaining Yelp Business IDs of Foodtrucks

# Setting Parameters
PARAMS = {'location': 'IN, US',
          'term': 'food truck',
          'categories': 'foodtrucks',
          'offset': 0}

# Scraping Data to Obtain Business ID List
response = requests.get(url = search_url, params = PARAMS, headers = Authorization)
bus_data = response.json()
df = pd.DataFrame(bus_data['businesses'])
id_list = list(df['id'])

In [11]:
# Scraping Data About Food Truck Details

# Creating Empty List to Store Scraped Data
dataframes = []

# Scraping Data of Details of Every Food Truck
for business_id in id_list:

    # Creating Header and Endpoint Variables
    headers = {"Authorization": f"Bearer {Yelp_API_Key}"}
    endpoint = f"https://api.yelp.com/v3/businesses/{business_id}"

    # Scraping Data
    response = requests.get(endpoint, headers=headers)
    if response.status_code == 200:
        business_data = response.json()

        # Obtaining Cuisine Data
        last_category_title = business_data['categories'][-1]['title']

        # Obtaining Start and End Time Hours
        if 'hours' in business_data:
          hours = business_data.get('hours')
          start_time = hours[0]['open'][0]['start']
          end_time = hours[0]['open'][0]['end']
        else:
          start_time = None
          end_time = None

        # Selecting Relevant Data from Business Details
        data = {
            "id": [business_data['id']],
            "name": [business_data['name']],
            "url": [business_data['url']],
            "display_phone": [business_data.get('display_phone')],
            "category": last_category_title,
            "rating": [business_data['rating']],
            "review_count": [business_data['review_count']],
            "latitude": [business_data['coordinates']['latitude']],
            "longitude": [business_data['coordinates']['longitude']],
            "start_time": start_time,
            "end_time": end_time
            }

        # Creating a DataFrame for the Business Details
        df = pd.DataFrame(data)
        dataframes.append(df)
    else:
        print(f"Failed to retrieve details for business with ID {business_id}")

# Concatenating the List of DataFrames into a Single DataFrame
result_df = pd.concat(dataframes, ignore_index=True)


In [12]:
# Viewing the Resulting DataFrame
display_result_df = result_df
html_1 = scroll_table(display_result_df, 'result_df_html', "Relevant Scraped Data")
display(HTML(html_1))

,id,name,url,display_phone,category,rating,review_count,latitude,longitude,start_time,end_time
0,r3MEtXbP52W_DL1pod5vkA,BB&B Bread Truck,https://www.yelp.com/biz/bb-and-b-bread-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 730-4778,Bakeries,4.5,38,39.779108,-86.163845,1700,2200
1,J2TZvGFySrShrpBQVxSWYg,Nacho Mama's Food Truck,https://www.yelp.com/biz/nacho-mamas-food-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 374-6969,Tex-Mex,4.5,20,39.771358,-86.156349,None,None
2,7ImnWM5f3pnFoi49Cp0xKQ,Tacos Without Borders,https://www.yelp.com/biz/tacos-without-borders-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(219) 508-7956,Food Trucks,4.5,23,39.691330,-86.075317,None,None
3,rUE8KrtSU8vrc_Cv8vnNuQ,Fat Cat's Bar-B-Que,https://www.yelp.com/biz/fat-cats-bar-b-que-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 523-1656,Barbeque,3.5,3,39.791033,-86.214710,None,None
4,gWrfmfVXy6EsNofL_8H2xQ,The Legendary Kitchen,https://www.yelp.com/biz/the-legendary-kitchen-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(800) 568-9370,Food Trucks,5.0,2,39.778465,-86.156783,0900,1700
5,_btYvQUAdnyDRh9I17PKDg,Caveman Truck,https://www.yelp.com/biz/caveman-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 674-3346,Food Trucks,4.0,26,39.768501,-86.158073,None,None
6,qFMx3Z_8-X0_osgIJAoZVg,Gaucho's Fire Express,https://www.yelp.com/biz/gauchos-fire-express-indianapolis-2?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 344-9921,Caterers,4.5,77,39.784037,-86.163322,1030,2100
7,7oTfGCHAqrtCZznNCI4rbA,Birrieria Iturbidense,https://www.yelp.com/biz/birrieria-iturbidense-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 828-3558,Tacos,4.5,40,39.747030,-86.270270,1800,0000
8,42LeGW-zeLbTgBkO0rJn9A,Serendipity Mobile Catering,https://www.yelp.com/biz/serendipity-mobile-catering-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 306-6870,Caterers,4.5,32,39.862871,-86.146483,0700,0100
9,er8PqnnrYifVI_7heMdVMA,Jumbo Jive Turkey Legs Smoked & Stuffed,https://www.yelp.com/biz/jumbo-jive-turkey-legs-smoked-and-stuffed-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 203-8484,Turkish,4.0,9,39.781132,-86.173824,None,None


In [14]:
# Extracting Formatted Addresses

# Creating Lists of Latitude and Longitude pairs
latitude_list = list(result_df['latitude'])
longitude_list = list(result_df['longitude'])

# Creating an Empty List to Store the Geocoded Addresses
geocoded_addresses = []

# Finding Formatted Addresss of Each Coordinate Pair
for i in range(len(latitude_list)):
    reverse_geocode_result = gmaps.reverse_geocode((latitude_list[i], longitude_list[i]))

    # Extracting the Formatted Address From the Result
    if reverse_geocode_result:
        formatted_address = reverse_geocode_result[0]['formatted_address']
        geocoded_addresses.append(formatted_address)
    else:
        geocoded_addresses.append("No address found for the given coordinates.")


In [15]:
# Creating a New DataFrame, Selecting Relevant Columns
new_df = result_df[['name', 'url', 'display_phone', 'category', 'rating', 'review_count', 'start_time', 'end_time', 'latitude', 'longitude']]
new_df['Address'] = geocoded_addresses



In [16]:
# Viewing the New DataFrame Containing the Formatted Addresses
display_new_df = new_df
html_2 = scroll_table(display_new_df, 'new_df_html', "Data Frame with Formatted Addresses")
display(HTML(html_2))

,name,url,display_phone,category,rating,review_count,start_time,end_time,latitude,longitude,Address
0,BB&B Bread Truck,https://www.yelp.com/biz/bb-and-b-bread-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 730-4778,Bakeries,4.5,38,1700,2200,39.779108,-86.163845,"800 Lafayette St, Indianapolis, IN 46202, USA"
1,Nacho Mama's Food Truck,https://www.yelp.com/biz/nacho-mamas-food-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 374-6969,Tex-Mex,4.5,20,None,None,39.771358,-86.156349,"58 E New York St, Indianapolis, IN 46204, USA"
2,Tacos Without Borders,https://www.yelp.com/biz/tacos-without-borders-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(219) 508-7956,Food Trucks,4.5,23,None,None,39.691330,-86.075317,"5407 Yucatan Dr, Indianapolis, IN 46237, USA"
3,Fat Cat's Bar-B-Que,https://www.yelp.com/biz/fat-cats-bar-b-que-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 523-1656,Barbeque,3.5,3,None,None,39.791033,-86.214710,"1832 N Moreland Ave, Indianapolis, IN 46222, USA"
4,The Legendary Kitchen,https://www.yelp.com/biz/the-legendary-kitchen-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(800) 568-9370,Food Trucks,5.0,2,0900,1700,39.778465,-86.156783,"40 E St Clair St, Indianapolis, IN 46204, USA"
5,Caveman Truck,https://www.yelp.com/biz/caveman-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 674-3346,Food Trucks,4.0,26,None,None,39.768501,-86.158073,"1861 Monument Cir, Indianapolis, IN 46204, USA"
6,Gaucho's Fire Express,https://www.yelp.com/biz/gauchos-fire-express-indianapolis-2?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 344-9921,Caterers,4.5,77,1030,2100,39.784037,-86.163322,"1280 N Senate Ave, Indianapolis, IN 46202, USA"
7,Birrieria Iturbidense,https://www.yelp.com/biz/birrieria-iturbidense-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 828-3558,Tacos,4.5,40,1800,0000,39.747030,-86.270270,"6302 W Washington St, Indianapolis, IN 46241, USA"
8,Serendipity Mobile Catering,https://www.yelp.com/biz/serendipity-mobile-catering-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 306-6870,Caterers,4.5,32,0700,0100,39.862871,-86.146483,"5922 N College Ave, Indianapolis, IN 46220, USA"
9,Jumbo Jive Turkey Legs Smoked & Stuffed,https://www.yelp.com/biz/jumbo-jive-turkey-legs-smoked-and-stuffed-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 203-8484,Turkish,4.0,9,None,None,39.781132,-86.173824,"910 W 10th St, Indianapolis, IN 46202, USA"


In [22]:
# Renaming the Columns, and Cleaning the DataFrame
new_col_names = {'name': 'Food_Truck',
                 'url': 'Website',
                 'display_phone': 'Phone_Number',
                 'category': 'Cuisine',
                 'rating': 'Rating',
                 'review_count': 'Review_Count',
                 'start_time': 'Start_Time',
                 'end_time': 'End_Time',
                 'hours': 'Hours',
                 'latitude': 'Latitude',
                 'longitude': 'Longitude',
                 'Address': 'Address'}
clean_df = new_df.rename(columns = new_col_names).dropna().drop_duplicates().reset_index(drop=True)

In [23]:
# Viewing the Cleaned DataFrame
display_clean_df = clean_df
html_3 = scroll_table(display_clean_df, 'clean_df_html', "Cleaned Data Frame")
display(HTML(html_3))

,Food_Truck,Website,Phone_Number,Cuisine,Rating,Review_Count,Start_Time,End_Time,Latitude,Longitude,Address
0,BB&B Bread Truck,https://www.yelp.com/biz/bb-and-b-bread-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 730-4778,Bakeries,4.5,38,1700,2200,39.779108,-86.163845,"800 Lafayette St, Indianapolis, IN 46202, USA"
1,The Legendary Kitchen,https://www.yelp.com/biz/the-legendary-kitchen-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(800) 568-9370,Food Trucks,5.0,2,0900,1700,39.778465,-86.156783,"40 E St Clair St, Indianapolis, IN 46204, USA"
2,Gaucho's Fire Express,https://www.yelp.com/biz/gauchos-fire-express-indianapolis-2?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 344-9921,Caterers,4.5,77,1030,2100,39.784037,-86.163322,"1280 N Senate Ave, Indianapolis, IN 46202, USA"
3,Birrieria Iturbidense,https://www.yelp.com/biz/birrieria-iturbidense-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 828-3558,Tacos,4.5,40,1800,0000,39.747030,-86.270270,"6302 W Washington St, Indianapolis, IN 46241, USA"
4,Serendipity Mobile Catering,https://www.yelp.com/biz/serendipity-mobile-catering-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 306-6870,Caterers,4.5,32,0700,0100,39.862871,-86.146483,"5922 N College Ave, Indianapolis, IN 46220, USA"
5,Fast Taco's,https://www.yelp.com/biz/fast-tacos-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 701-8182,Food Trucks,4.5,16,1900,0100,39.867496,-86.202026,"6154 Michigan Rd, Indianapolis, IN 46228, USA"
6,Taqueria Oaxaca Indy,https://www.yelp.com/biz/taqueria-oaxaca-indy-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 490-2429,Mexican,4.5,9,1900,0100,39.822687,-86.235794,"4501 W 38th St, Indianapolis, IN 46254, USA"
7,Black Leaf Vegan,https://www.yelp.com/biz/black-leaf-vegan-indianapolis-2?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 560-4222,Comfort Food,4.0,65,0800,1800,39.779130,-86.164590,"335 W 9th St, Indianapolis, IN 46202, USA"
8,Poccadio Grill Food Truck,https://www.yelp.com/biz/poccadio-grill-food-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 252-5911,Food Trucks,4.0,13,1100,2300,39.866074,-86.101631,"6116 N Chester Ave, Indianapolis, IN 46220, USA"
9,Greiner's,https://www.yelp.com/biz/greiners-indianapolis-3?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 783-4136,Food Trucks,3.5,118,1030,2000,39.738850,-86.140100,"2126 Shelby St, Indianapolis, IN 46203, USA"


In [24]:
# Converting Hours to Date_Time Format

# Importing Date_Time Package
from datetime import datetime
from math import nan


# Creating Lists to Store Start and End Times
start_times = list(clean_df['Start_Time'])
end_times = list(clean_df['End_Time'])

# Converting Start Times to Date_Time Format
start_times_datetime = [datetime.strptime(time, '%H%M') if time is not nan else nan for time in start_times]

# Convert End Times to Date_Time Format
end_times_datetime = [datetime.strptime(time, '%H%M') if time is not nan else nan for time in end_times]

# Create a DataFrame
time_df = pd.DataFrame({'Start_Time': start_times_datetime, 'End_Time': end_times_datetime})

# Format the Date_Time Objects in the Desired Format
time_df['Start_Time'] = time_df['Start_Time'].dt.strftime('%I:%M %p')
time_df['End_Time'] = time_df['End_Time'].dt.strftime('%I:%M %p')


In [25]:
# Adding Start and End Times to Clean DataFrame

# Dropping Hours Column
clean_df = clean_df.drop(columns=['Start_Time', 'End_Time'])

# Adding Start and End Time Columns for Planning DataFrame
clean_df['Start_Time'] = time_df['Start_Time']
clean_df['End_Time'] = time_df['End_Time']

# Adding Operating Hours Column for Final DataFrame
clean_df['Operating_Hours'] = clean_df['Start_Time'] + ' - ' + clean_df['End_Time']

In [26]:
# Viewing the Clean DataFrame Containing the Operating Hours
display_clean_times_df = clean_df
html_4 = scroll_table(display_clean_times_df, 'final_df_html', "Clean Data Frame with Operating Hours")
display(HTML(html_4))

,Food_Truck,Website,Phone_Number,Cuisine,Rating,Review_Count,Latitude,Longitude,Address,Start_Time,End_Time,Operating_Hours
0,BB&B Bread Truck,https://www.yelp.com/biz/bb-and-b-bread-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 730-4778,Bakeries,4.5,38,39.779108,-86.163845,"800 Lafayette St, Indianapolis, IN 46202, USA",05:00 PM,10:00 PM,05:00 PM - 10:00 PM
1,The Legendary Kitchen,https://www.yelp.com/biz/the-legendary-kitchen-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(800) 568-9370,Food Trucks,5.0,2,39.778465,-86.156783,"40 E St Clair St, Indianapolis, IN 46204, USA",09:00 AM,05:00 PM,09:00 AM - 05:00 PM
2,Gaucho's Fire Express,https://www.yelp.com/biz/gauchos-fire-express-indianapolis-2?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 344-9921,Caterers,4.5,77,39.784037,-86.163322,"1280 N Senate Ave, Indianapolis, IN 46202, USA",10:30 AM,09:00 PM,10:30 AM - 09:00 PM
3,Birrieria Iturbidense,https://www.yelp.com/biz/birrieria-iturbidense-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 828-3558,Tacos,4.5,40,39.747030,-86.270270,"6302 W Washington St, Indianapolis, IN 46241, USA",06:00 PM,12:00 AM,06:00 PM - 12:00 AM
4,Serendipity Mobile Catering,https://www.yelp.com/biz/serendipity-mobile-catering-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 306-6870,Caterers,4.5,32,39.862871,-86.146483,"5922 N College Ave, Indianapolis, IN 46220, USA",07:00 AM,01:00 AM,07:00 AM - 01:00 AM
5,Fast Taco's,https://www.yelp.com/biz/fast-tacos-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 701-8182,Food Trucks,4.5,16,39.867496,-86.202026,"6154 Michigan Rd, Indianapolis, IN 46228, USA",07:00 PM,01:00 AM,07:00 PM - 01:00 AM
6,Taqueria Oaxaca Indy,https://www.yelp.com/biz/taqueria-oaxaca-indy-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 490-2429,Mexican,4.5,9,39.822687,-86.235794,"4501 W 38th St, Indianapolis, IN 46254, USA",07:00 PM,01:00 AM,07:00 PM - 01:00 AM
7,Black Leaf Vegan,https://www.yelp.com/biz/black-leaf-vegan-indianapolis-2?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 560-4222,Comfort Food,4.0,65,39.779130,-86.164590,"335 W 9th St, Indianapolis, IN 46202, USA",08:00 AM,06:00 PM,08:00 AM - 06:00 PM
8,Poccadio Grill Food Truck,https://www.yelp.com/biz/poccadio-grill-food-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 252-5911,Food Trucks,4.0,13,39.866074,-86.101631,"6116 N Chester Ave, Indianapolis, IN 46220, USA",11:00 AM,11:00 PM,11:00 AM - 11:00 PM
9,Greiner's,https://www.yelp.com/biz/greiners-indianapolis-3?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 783-4136,Food Trucks,3.5,118,39.738850,-86.140100,"2126 Shelby St, Indianapolis, IN 46203, USA",10:30 AM,08:00 PM,10:30 AM - 08:00 PM


In [28]:
# Creating Final Dataset, After Further Cleaning and Sorting by Rating
final_df = clean_df.dropna().sort_values(by='Rating', ascending=False).reset_index(drop=True).drop(columns=['Latitude', 'Longitude',
                                                                                                   'Start_Time', 'End_Time'])

# Creating a Dataset with Information valuable for the Plan Formulation
plan_df = clean_df.dropna().sort_values(by='Rating', ascending=False).reset_index(drop=True).drop(columns=['Operating_Hours'])

In [29]:
# Viewing the Final DataFrame
display_final_df = final_df
html_5 = scroll_table(display_final_df, 'final_df_html', "Final Data Frame")
display(HTML(html_5))

,Food_Truck,Website,Phone_Number,Cuisine,Rating,Review_Count,Address,Operating_Hours
0,La Familia,https://www.yelp.com/biz/la-familia-food-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(463) 280-4499,Tacos,5.0,8,"311 Troy Ct, Indianapolis, IN 46227, USA",11:00 AM - 11:00 PM
1,Tapatia Mexican Tacos,https://www.yelp.com/biz/tapatia-mexican-tacos-warren-park?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 490-2417,Tacos,5.0,1,"7202 E Washington St, Indianapolis, IN 46219, USA",12:00 PM - 11:00 PM
2,Ice Cream On Wheels,https://www.yelp.com/biz/ice-cream-on-wheels-indianapolis-6?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(800) 884-9793,Food Trucks,5.0,1,"4215 W Thompson Rd, Indianapolis, IN 46221, USA",12:00 AM - 12:00 AM
3,Traveling Taste Buds,https://www.yelp.com/biz/traveling-taste-buds-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 531-6155,Food Trucks,5.0,1,"1 Monument Cir, Indianapolis, IN 46204, USA",12:00 AM - 12:00 AM
4,HomeTeam Ice Cream,https://www.yelp.com/biz/hometeam-ice-cream-kokomo?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(765) 437-6843,Food Trucks,5.0,2,"709 N Union St, Kokomo, IN 46901, USA",04:00 PM - 08:00 PM
5,TACO LOCO M&C,https://www.yelp.com/biz/taco-loco-m-and-c-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(463) 210-9667,Food Trucks,5.0,1,"1415 W 86th St, Indianapolis, IN 46260, USA",06:00 PM - 12:00 AM
6,Tacopolis,https://www.yelp.com/biz/tacopolis-indianapolis-2?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 488-8724,Tacos,5.0,5,"4857 W 38th St, Indianapolis, IN 46254, USA",02:00 PM - 11:30 PM
7,The Legendary Kitchen,https://www.yelp.com/biz/the-legendary-kitchen-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(800) 568-9370,Food Trucks,5.0,2,"40 E St Clair St, Indianapolis, IN 46204, USA",09:00 AM - 05:00 PM
8,Scout's Treat Truck,https://www.yelp.com/biz/scouts-treat-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 409-2600,Cupcakes,4.5,19,"8375 Allison Pointe Blvd, Indianapolis, IN 46250, USA",06:30 AM - 08:00 PM
9,Bearded Bagel,https://www.yelp.com/biz/bearded-bagel-indianapolis-3?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 516-5938,Sandwiches,4.5,101,"7305 E 96th St #400, Indianapolis, IN 46250, USA",07:00 AM - 02:00 PM


In [30]:
# Exporting Final DataFrame to Excel
final_df.to_excel('Final_Cleaned_DataFrame.xlsx', index=False)

# Exporting Plan DataFrame to Excel
plan_df.to_excel('Plan_df.xlsx', index=False)

In [31]:
# Downloading Excel Files
files.download('Final_Cleaned_DataFrame.xlsx')
files.download('Plan_df.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>